In [1]:
# Import necesary libraries for data import/analysis
import pandas as pd
import numpy as np
import scipy
from scipy.io import loadmat

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Importing necessary libraries for visualization
import plotly # plotting engine
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio # export plot as interactive html
import kaleido # export plot as high quality vector graphics
import shelve #for importing atlas vis data from db file

In [3]:
# Structure-Function Derived Metrics

#=.mat file input into MATLAB script
#Desired structure:
#.mat file containing
# fc 690*690, functional similarity coefficients
# sc 690*690, streamline count (average of 1065 used since matlab script expects dense sc matrix)
# d 690*690, Euclidean distance between parcel centroids
# hemi 690*1, 1 for left and 2 for right hemisphere
# nbins, scalar = 31

#Predictor key

# a = structural connectivity
# abin = binarised structural connectivity
# number of streamlines ending in pairs of nodes averaged across 1065 subjects

# d = euclidean distance between centroids
# Euclidean distance between regional centroids

# PLbin = binarised path length
# PLwei = weighted path length (1,2,3,4 => gamma values of 0.25, 0.50, 1, & 2 in cost function)
# Here, we calculated shortest paths matrices for the binary network (where the cost is identical for all existing edges) and also for a parameterized affinity-to-cost transformation evaluated at several different parameter values.

# Gwei = weighted communicability
# Gbin = binarised communicability **this derived metric failed
# Communicability is a weighted sum of walks of all lengths between pairs of nodes.

# Coswei = weighted cosine distance
# Cosbin = binarised cosine distance
# The cosine similarity measures the angle between two vectors

# SIbin = search information binarised
# SIwei = weighted search information (1,2,3,4 => gamma values of 0.25, 0.50, 1, & 2 in cost function)
# Search information measures the amount of information (in bits) required to traverse shortest paths in a network

# PTbin = binarised path transitivity
# PTbwei = weighted path transitivity

# MIwei = weighted matching index
# MIbin = binarised matching index
# The matching index is a measure of overlap between pairs of nodes based on their connectivity profiles.

# Navnumhops = navigability in number of hops
# NavplMS = navigability in path length metric space
# One such routing rule is, given a target node T, to always move towards the node nearest the target in some metric space, e.g. Euclidean space.

# mfptwei = weighted mean first passage time
# mfptbin = binarised mean first passage time
# The mean first passage time (MFPT) refers to the expected number of steps a random walk must evolve for a random walked starting at node i to end up at node j

# FGbin = binarised flow graph (1,2,3,4 => markov times of 1, 2, 5, & 10)
# FGwei = weighted flow graph (1,2,3,4 => markov times of 1, 2, 5, & 10)
# A flow graph is a transformation of a network’s (possibly sparse) connectivity matrix, Aij, into a fully-weighted matrix in which the dynamics of a Markov process are embedded into edge weights

In [7]:
# Import rsq matrix (predictor by parcel)
# generated from compute_structural_connectivity_metrics.m
rsq = loadmat('../data/connectivity/optimized_sc/optimized_sc_rsq.mat')
rsq = np.array(rsq['rsq'])
rsq[np.isnan(rsq)] = 0
col_names = ['a','abin','d','PLbin','PLwei1','PLwei2','PLwei3','PLwei4','Gwei','Gbin','Coswei','Cosbin','SIbin','SIwei1','SIwei2','SIwei3','SIwei4','PTbin','PTwei1','PTwei2','PTwei3','PTwei4','MIwei','MIbin','Navnumhops','NavplMS','mfptwei','mfptbin','FGbin1','FGbin2','FGbin3','FGbin4','FGwei1','FGwei2','FGwei3','FGwei4']
rsq_df = pd.DataFrame(rsq, columns = col_names)

# Import predictors
# generated from compute_structural_connectivity_metrics.m
predictors = loadmat('../data/connectivity/optimized_sc/optimized_sc_predictors.mat')
pred_dict = dict.fromkeys(col_names)
for pred in col_names:
    try:
        pred_dict[pred] = predictors[pred]
    except KeyError:
        try:
            temp_pred = pred[:-1]
            pred_dict[pred] = predictors[temp_pred][:,:,int(pred[-1])-1]
        except:
            print("Failed: ", pred)
            continue

# Import yba dataframe (690 parcel version)
parcel_dict = pd.read_csv('../data/misc/parcel_dict.csv')

# Inititalise atlas variables
atlas_labels = parcel_dict["Name"].to_list()
atlas_gyrus = parcel_dict["Gyrus"].to_list()
atlas_region = parcel_dict["Region"].to_list()
atlas_lobe = parcel_dict["Lobe"].to_list()

In [8]:
best_pred = np.argmax(rsq,1)
i = 0
best_pred_list = []
best_pred_rval = []
for predictor in best_pred:
    best_pred_list.append(col_names[predictor])
    best_pred_rval.append(rsq_df[col_names[predictor]][i])
    i = i+1

In [10]:
sf_df = pd.DataFrame()
sf_df["Parcel"] = atlas_labels
sf_df["Region"] = atlas_region
sf_df["Best Predictor"] = best_pred_list
sf_df["Best Predictor Rsquare"] = best_pred_rval
for predictor in col_names:
    sf_df[predictor] = rsq_df[predictor].tolist()

In [11]:
sf_df.to_csv('../data/connectivity/optimized_sc/sc_metrics_sf_analysis.csv', index=False)
sf_df

,Parcel,Region,Best Predictor,Best Predictor Rsquare,a,abin,d,PLbin,PLwei1,PLwei2,...,mfptwei,mfptbin,FGbin1,FGbin2,FGbin3,FGbin4,FGwei1,FGwei2,FGwei3,FGwei4
0,L_TP1_A,Temporal pole,d,0.256443,0.079562,0.017671,0.256443,0.017671,0.000908,0.006742,...,0.153514,0.046002,0.009648,0.000004,0.016456,0.026662,0.070569,0.036591,0.000037,0.050232
1,L_TP1_B,Temporal pole,d,0.289186,0.027674,0.016591,0.289186,0.016591,0.000531,0.019104,...,0.118389,0.054070,0.009166,0.000008,0.013174,0.020464,0.023729,0.010416,0.001758,0.089312
2,L_TP1_C,Temporal pole,d,0.102871,0.000406,0.002590,0.102871,0.002590,0.001202,0.003651,...,0.008822,0.000071,0.004123,0.006766,0.007378,0.007245,0.001219,0.001051,0.000893,0.038590
3,L_TP1_D,Temporal pole,MIwei,0.102033,0.007391,0.005263,0.072611,0.005263,0.003681,0.000044,...,0.012379,0.000250,0.008255,0.012028,0.011703,0.011176,0.006241,0.002774,0.000399,0.024923
4,L_TP1_E,Temporal pole,MIwei,0.040245,0.020545,0.008161,0.032463,0.008161,0.000023,0.000045,...,0.003581,0.000409,0.009945,0.010429,0.008312,0.007804,0.015026,0.005585,0.000140,0.011296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,R_I4_D,Insula,d,0.069039,0.007822,0.005187,0.069039,0.005187,0.020130,0.010024,...,0.006996,0.001114,0.012092,0.033390,0.054214,0.053244,0.004395,0.000198,0.007017,0.036288
686,R_I5_A,Insula,FGwei2,0.194770,0.133289,0.009973,0.032505,0.009973,0.055047,0.031185,...,0.062507,0.013878,0.008324,0.003413,0.000255,0.001642,0.170469,0.194770,0.143619,0.058605
687,R_I5_B,Insula,FGwei1,0.210807,0.197748,0.001874,0.029941,0.001874,0.050881,0.016780,...,0.086391,0.004383,0.000878,0.000040,0.003005,0.004241,0.210807,0.197853,0.123449,0.040171
688,R_I5_C,Insula,d,0.034323,0.002442,0.004048,0.034323,0.004048,0.004925,0.000083,...,0.004381,0.000647,0.007817,0.015406,0.018183,0.016991,0.001618,0.000174,0.002672,0.017844
